1. diabetes food hub information

2. daily diets
1) sampling using maximal marginal relevance search algorithm
2) train/test split
3) paste the instruction

In [1]:
import pandas as pd

dfh = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub.csv")

In [2]:
dfh.drop_duplicates(['title'], inplace=True)

In [3]:
dfh.columns

Index(['Unnamed: 0', 'title', 'description', 'prep_time', 'cook_time',
       'servings', 'steps', 'tags', 'ingredients',
       'nutrition_facts.Amount per serving', 'nutrition_facts.Total Fat',
       'nutrition_facts.Cholesterol', 'nutrition_facts.Sodium',
       'nutrition_facts.Total Carbohydrate', 'nutrition_facts.Protein',
       'file_name'],
      dtype='object')

In [4]:
dfh.head(2)

,Unnamed: 0,title,description,prep_time,cook_time,servings,steps,tags,ingredients,nutrition_facts.Amount per serving,nutrition_facts.Total Fat,nutrition_facts.Cholesterol,nutrition_facts.Sodium,nutrition_facts.Total Carbohydrate,nutrition_facts.Protein,file_name
0,0,Chicken Apple Crunch Salad,This savory and sweet chicken apple crunch sal...,5 min,1 hr,5,['Cube cooked chicken. Dice apple and celery. ...,"['CKD Non-Dialysis', 'CKD Dialysis', 'Kidney-F...","[{'label': 'cooked chicken', 'us_measure': '2 ...",230,10g,65mg,210mg,15g,21g,kidney-friendly
1,1,Broccoli and Apple Salad,This kidney-friendly recipe is a kid favorite....,10 min,1 hr,8,['Add the remaining ingredients and coat with ...,"['CKD Non-Dialysis', 'CKD Dialysis', 'Kidney-F...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me...",130,9g,9mg,70mg,12g,4g,kidney-friendly


In [5]:
len(dfh)

1336

In [6]:
import pandas as pd
import ast
import json

df = dfh

def create_instruction_dataset_from_row(row):
    instructions = []

    try:
        nutrition_facts = {
            "Amount per serving": row["nutrition_facts.Amount per serving"],
            "Total Fat": row["nutrition_facts.Total Fat"],
            "Cholesterol": row["nutrition_facts.Cholesterol"],
            "Sodium": row["nutrition_facts.Sodium"],
            "Total Carbohydrate": row["nutrition_facts.Total Carbohydrate"],
            "Protein": row["nutrition_facts.Protein"],
        }
        nutrition_output = ", ".join([f"{key}: {value}" for key, value in nutrition_facts.items()])
    except Exception as e:
        print(f"Error parsing nutrition facts: {e}")
        nutrition_output = "N/A"

    instructions.append({
        "instruction": "Provide the nutrition facts for the given recipe.",
        "input": row["title"],
        "output": nutrition_output,
    })

    try:
        ingredients = ast.literal_eval(row["ingredients"])
        if isinstance(ingredients, list) and all(isinstance(ing, dict) for ing in ingredients):
            ingredients_output = "\n".join(
                [f"- {ing['label']}: {ing.get('us_measure', 'N/A')} ({ing.get('metric_measure', 'N/A')})" for ing in ingredients]
            )
        else:
            ingredients_output = "N/A"
    except Exception as e:
        print(f"Error parsing ingredients: {row['ingredients']} - {e}")
        ingredients_output = "N/A"

    instructions.append({
        "instruction": "List the measurements for all ingredients in the given recipe.",
        "input": row["title"],
        "output": ingredients_output,
    })

    try:
        tags = ast.literal_eval(row["tags"])
        if isinstance(tags, list):
            tags_output = ", ".join(tags)
        else:
            tags_output = "N/A"
    except Exception as e:
        print(f"Error parsing tags: {row['tags']} - {e}")
        tags_output = "N/A"

    instructions.append({
        "instruction": "What tags are associated with this recipe?",
        "input": row["title"],
        "output": tags_output,
    })

    try:
        description_output = row["description"] if isinstance(row["description"], str) else "N/A"
    except Exception as e:
        print(f"Error parsing description: {e}")
        description_output = "N/A"

    instructions.append({
        "instruction": "Provide a brief description of the given recipe.",
        "input": row["title"],
        "output": description_output,
    })

    return instructions

dfh_instruction_dataset = []

for index, row in df.iterrows():
    print(f"Processing row {index}...") 
    dfh_instruction_dataset.extend(create_instruction_dataset_from_row(row))


Processing row 0...
Processing row 1...
Processing row 2...
Processing row 3...
Processing row 4...
Processing row 5...
Processing row 6...
Processing row 7...
Processing row 8...
Processing row 9...
Processing row 10...
Processing row 31...
Processing row 32...
Processing row 33...
Processing row 34...
Processing row 35...
Processing row 36...
Processing row 37...
Processing row 48...
Processing row 49...
Processing row 50...
Processing row 51...
Processing row 52...
Processing row 53...
Processing row 54...
Processing row 55...
Processing row 56...
Processing row 67...
Processing row 68...
Processing row 69...
Processing row 70...
Processing row 71...
Processing row 72...
Processing row 73...
Processing row 74...
Processing row 75...
Processing row 86...
Processing row 87...
Processing row 88...
Processing row 89...
Processing row 90...
Processing row 91...
Processing row 92...
Processing row 93...
Processing row 94...
Processing row 105...
Processing row 106...
Processing row 107...

In [7]:
dfh_instruction_dataset[0:4]

[{'instruction': 'Provide the nutrition facts for the given recipe.',
  'input': 'Chicken Apple Crunch Salad',
  'output': 'Amount per serving: 230, Total Fat: 10g, Cholesterol: 65mg, Sodium: 210mg, Total Carbohydrate: 15g, Protein: 21g'},
 {'instruction': 'List the measurements for all ingredients in the given recipe.',
  'input': 'Chicken Apple Crunch Salad',
  'output': '- cooked chicken: 2 cup (473 ml)\n- Gala apples: 1 cup (237 ml)\n- celery: 1/2 cup (118 ml)\n- scallions: 2 tbsp (30 ml)\n- raisins: 1/4 cup (59 ml)\n- light mayonnaise: 1/3 cup (78 ml)\n- light sour cream: 1 tbsp (15 ml)\n- lemon juice: 1 tbsp (15 ml)\n- ground cinnamon: 1/4 tsp (1 g)\n- black pepper: 1/4 tsp (1 g)'},
 {'instruction': 'What tags are associated with this recipe?',
  'input': 'Chicken Apple Crunch Salad',
  'output': 'CKD Non-Dialysis, CKD Dialysis, Kidney-Friendly, Kid Friendly, Main Dish, Budget Friendly, Dinner, Lunch, Quick & Easy'},
 {'instruction': 'Provide a brief description of the given reci

In [8]:
import pandas as pd

dfh_instruction_dataset = pd.DataFrame(dfh_instruction_dataset)

# dfh_instruction_dataset.to_json("src/data/data2_daily_diets/dfh_instrution_dataset.json")

In [ ]:
# import os
# import json

# directory_path = "src/data/data2_daily_diets/weekly_diet_with_ingredients_score"
# output_file = "daily_diets.json" 

# merged_data = []

# for file_name in os.listdir(directory_path):
#     if file_name.endswith(".json"):
#         file_path = os.path.join(directory_path, file_name)
        
#         with open(file_path, "r", encoding="utf-8") as f:
#             try:
#                 data = json.load(f)

#                 evaluation = data.get("Evaluation", {})
#                 score = evaluation.get("Score")

#                 if score in [3, 4, 5]:
                    
#                     for day_key in [f"Day {i}" for i in range(1, 8)]:
#                         if day_key in data:
#                             day_data = data[day_key]
#                             merged_data.append(day_data)
#             except json.JSONDecodeError:
#                 print(f"Error decoding JSON in file: {file_name}")

# # 6. 결과 저장
# with open(output_file, "w", encoding="utf-8") as output:
#     json.dump(merged_data, output, ensure_ascii=False, indent=4)

# print(f"Filtered and merged data saved to {output_file}")


Filtered and merged data saved to daily_diets.json


In [ ]:
# import json

# def remove_duplicates_from_json(input_file, output_file):

#     with open(input_file, "r") as f:
#         data = json.load(f)

#     seen = set()
#     unique_data = []

#     for day_data in data:
#         day_data_str = json.dumps(day_data, sort_keys=True)
#         if day_data_str not in seen:
#             seen.add(day_data_str)
#             unique_data.append(day_data)

#     with open(output_file, "w") as f:
#         json.dump(unique_data, f, ensure_ascii=False, indent=2)

#     print(f"Duplicates removed. Cleaned data saved to {output_file}")

# input_file = "/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets.json" 
# output_file = "daily_diets_drop_duplicated.json" 
# remove_duplicates_from_json(input_file, output_file)


Duplicates removed. Cleaned data saved to daily_diets2.json


In [ ]:
# with open("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated.json", 'r', encoding='utf-8') as f:
#     daily_diets = json.load(f)

In [ ]:
# daily_diets[0]

{'Breakfast': {'Dish': 'Strawberry Cream Cheese French Toast Casserole',
  'Nutrition': {'Carbohydrate (g)': 27,
   'Protein (g)': 11,
   'Fat (g)': 12,
   'Sodium (mg)': 410,
   'Cholesterol (mg)': 165},
  'Ingredients': [{'label': 'Texas toast style bread',
    'metric_measure': '12 slice'},
   {'label': 'light cream cheese(cold)', 'metric_measure': '340 ml'},
   {'label': 'Fresh Strawberries', 'metric_measure': '473 ml'},
   {'label': 'eggs', 'metric_measure': '9 large'},
   {'label': 'fat-free half-and-half', 'metric_measure': '355 ml'},
   {'label': 'sugar-free maple-type syrup', 'metric_measure': '78 ml'},
   {'label': 'no calorie granulated sugar substitute (sucralose original)',
    'metric_measure': '177 ml'},
   {'label': 'unsalted butter', 'metric_measure': '10 g'}]},
 'Lunch': {'Dish': 'Grilled Strawberries on “Crème” with Balsamic Glaze',
  'Nutrition': {'Carbohydrate (g)': 15,
   'Protein (g)': 9,
   'Fat (g)': 8,
   'Sodium (mg)': 60,
   'Cholesterol (mg)': 10},
  'Ingre

In [10]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from tqdm import tqdm
import time

input_file = "/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated.json"
output_file = "daily_diets_drop_duplicated_and_sampled.json"

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json_line_by_line(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for entry in data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

def mmr_selection(documents, n_samples, lambda_param=0.5):
    """Select n_samples using Maximal Marginal Relevance (MMR)."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    selected_indices = []
    remaining_indices = list(range(len(documents)))

    selected_indices.append(remaining_indices.pop(0))

    for _ in tqdm(range(n_samples - 1), desc="Selecting samples"):
        max_mmr_score = -float('inf')
        best_idx = -1
        
        for idx in remaining_indices:
            relevance = np.mean([similarity_matrix[idx][i] for i in selected_indices])
            diversity = np.min([similarity_matrix[idx][i] for i in selected_indices])
            
            mmr_score = lambda_param * relevance - (1 - lambda_param) * diversity
            if mmr_score > max_mmr_score:
                max_mmr_score = mmr_score
                best_idx = idx

        selected_indices.append(best_idx)
        remaining_indices.remove(best_idx)

    return selected_indices

def process_json(input_file, output_file, n_samples=5000):
    data = load_json(input_file)

    documents = []
    for day_plan in data:
        documents.append(json.dumps(day_plan, ensure_ascii=False))

    print("Calculating MMR...")
    start_time = time.time()
    selected_indices = mmr_selection(documents, n_samples)
    end_time = time.time()

    elapsed_time = end_time - start_time
    print(f"MMR calculation completed in {elapsed_time:.2f} seconds.")

    sampled_data = [data[i] for i in selected_indices]

    print("Saving selected samples...")
    save_json_line_by_line(sampled_data, output_file)
    print(f"Saved {n_samples} samples to {output_file}")

if __name__ == "__main__":
    process_json(input_file, output_file)


Calculating MMR...


Selecting samples: 100%|██████████| 4999/4999 [14:31:15<00:00, 10.46s/it]  


MMR calculation completed in 52292.90 seconds.
Saving selected samples...
Saved 5000 samples to daily_diets_drop_duplicated_and_sampled.json


In [9]:
file_path = "/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated_and_sampled.json"
daily_diets = []
with open(file_path , 'r', encoding='utf-8') as f:
    for line in f:
        daily_diets.append(json.loads(line.strip()))

In [10]:
# modify the nutrition facts 

import pandas as pd
dfh2 = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub_new_nutri_facts.csv") 

In [11]:
dfh2.head(2)

,title,description,prep_time,cook_time,servings,steps,tags,nutrition_facts,ingredients
0,Raspberry Swirl Frozen Yogurt Bark,Raspberry Swirl Frozen Yogurt Bark: Dive into ...,10 min,4 hr,6 Servings,['Cover a freezer-safe tray with parchment pap...,"['Kid Friendly', 'Vegetarian', 'Dessert', 'Sna...","{'Servings': '6 Servings', 'Serving Size': '1 ...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me..."
1,Maple-Pumpkin Spice Oatmeal Cookies,Description not found,10 min,25 min,14 Servings,['Preheat the oven to 350 degrees F. Line two ...,"['Kid Friendly', 'Vegetarian', 'Snacks', 'Glut...","{'Servings': '14 Servings', 'Serving Size': '1...","[{'label': 'old-fashioned rolled oats', 'us_me..."


In [ ]:
import pandas as pd
import json
import ast

json_file_path = "/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated_and_sampled.json"

merged_df = dfh2

with open(json_file_path, 'r', encoding='utf-8') as f:
    daily_diets = [json.loads(line.strip()) for line in f]

def calculate_daily_total(diet):
    daily_total = {
        "Calories": 0,
        "Carbohydrate (g)": 0,
        "Fiber (g)": 0,
        "Protein (g)": 0,
        "Fat (g)": 0,
        "Cholesterol (mg)": 0,
        "Sodium (mg)": 0,
        "Potassium (mg)": 0
    }
    
    for meal_type in ["Breakfast", "Lunch", "Dinner"]:
        if meal_type in diet and "Nutrition" in diet[meal_type]:
            nutrition = diet[meal_type]["Nutrition"]
            
            # Calories
            calories = nutrition.get("Calories", "0").replace("g", "").replace("mg", "")
            daily_total["Calories"] += float(calories) if calories.isdigit() else 0
            
            # Carbohydrate (g)
            carbohydrate = nutrition.get("Total Carbohydrate", {}).get("Amount", "0g").replace("g", "")
            daily_total["Carbohydrate (g)"] += float(carbohydrate) if carbohydrate.isdigit() else 0

            # Fiber 
            fiber = nutrition.get("Total Carbohydrate", {}).get("Dietary Fiber", "0g").replace("g", "")
            daily_total["Fiber (g)"] += float(fiber) if fiber.isdigit() else 0
            
            # Protein (g)
            protein = nutrition.get("Protein", "0g").replace("g", "")
            daily_total["Protein (g)"] += float(protein) if protein.isdigit() else 0
            
            # Fat (g)
            fat = nutrition.get("Total Fat", {}).get("Amount", "0g").replace("g", "")
            daily_total["Fat (g)"] += float(fat) if fat.isdigit() else 0
            
            # Cholesterol (mg)
            cholesterol = nutrition.get("Cholesterol", "0mg").replace("mg", "")
            daily_total["Cholesterol (mg)"] += float(cholesterol) if cholesterol.isdigit() else 0
            
            # Sodium (mg)
            sodium = nutrition.get("Sodium", "0mg").replace("mg", "")
            daily_total["Sodium (mg)"] += float(sodium) if sodium.isdigit() else 0
            
            # Potassium (mg)
            potassium = nutrition.get("Potassium", "0mg")
            if potassium is None:
                potassium = "0mg"
            potassium = potassium.replace("mg", "")
            daily_total["Potassium (mg)"] += float(potassium) if potassium.isdigit() else 0
    
    return daily_total     

def parse_nutrition_facts(row):
    try:
        if isinstance(row["nutrition_facts"], str):
            nutrition_facts = ast.literal_eval(row["nutrition_facts"])
        elif isinstance(row["nutrition_facts"], dict):
            nutrition_facts = row["nutrition_facts"]
        else:
            nutrition_facts = {}

        amount_per_serving = nutrition_facts.get("Amount per Serving", {})
        nutrition_facts_parsed = {
            "Calories": amount_per_serving.get("Calories", "N/A"),
            "Total Fat": amount_per_serving.get("Total Fat", {}),
            "Cholesterol": amount_per_serving.get("Cholesterol", "N/A"),
            "Sodium": amount_per_serving.get("Sodium", "N/A"),
            "Total Carbohydrate": amount_per_serving.get("Total Carbohydrates", {}),
            "Protein": amount_per_serving.get("Protein", "N/A"),
            "Potassium": amount_per_serving.get("Potassium", "N/A"),
        }
        return nutrition_facts_parsed
    except Exception as e:
        print(f"Error parsing nutrition facts: {e}")
        return {}

def update_json_from_merged_df(df, daily_diets):
    for diet in daily_diets:
        for meal_type in ["Breakfast", "Lunch", "Dinner"]:
            if meal_type in diet:
                dish_name = diet[meal_type]["Dish"]

                matching_row = df[df["title"] == dish_name]
                if not matching_row.empty:
                    row = matching_row.iloc[0]
                    nutrition_facts = parse_nutrition_facts(row)
                    
                    diet[meal_type]["Nutrition"] = nutrition_facts
        
        diet["Daily Total"] = calculate_daily_total(diet)
    
    return daily_diets

updated_daily_diets = update_json_from_merged_df(merged_df, daily_diets)

output_path = "daily_diets_drop_duplicated_and_sampled_and_modified_nutri_facts.json"
with open(output_path, 'w', encoding='utf-8') as f:
    for diet in updated_daily_diets:
        f.write(json.dumps(diet, ensure_ascii=False) + "\n")

print(f"Updated JSON data saved to {output_path}")


Updated JSON data saved to daily_diets_drop_duplicated_and_sampled_and_modified_nutri_facts.json


In [13]:
file_path = "/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated_and_sampled_and_modified_nutri_facts.json"
daily_diets = []
with open(file_path , 'r', encoding='utf-8') as f:
    for line in f:
        daily_diets.append(json.loads(line.strip()))

In [14]:
daily_diets[0]

{'Breakfast': {'Dish': 'Strawberry Cream Cheese French Toast Casserole',
  'Nutrition': {'Calories': '260',
   'Total Fat': {'Amount': '12g',
    'Saturated Fat': '5.4g',
    'Trans Fat': '0.1g'},
   'Cholesterol': '165mg',
   'Sodium': '410mg',
   'Total Carbohydrate': {'Amount': '27g',
    'Dietary Fiber': '2g',
    'Total Sugars': '6g',
    'Added Sugars': '2g'},
   'Protein': '11g',
   'Potassium': '230mg'},
  'Ingredients': [{'label': 'Texas toast style bread',
    'metric_measure': '12 slice'},
   {'label': 'light cream cheese(cold)', 'metric_measure': '340 ml'},
   {'label': 'Fresh Strawberries', 'metric_measure': '473 ml'},
   {'label': 'eggs', 'metric_measure': '9 large'},
   {'label': 'fat-free half-and-half', 'metric_measure': '355 ml'},
   {'label': 'sugar-free maple-type syrup', 'metric_measure': '78 ml'},
   {'label': 'no calorie granulated sugar substitute (sucralose original)',
    'metric_measure': '177 ml'},
   {'label': 'unsalted butter', 'metric_measure': '10 g'}]},

In [1]:
# Secondary sampling using nutrition facts ratio

import json

def filter_diets_by_macros(daily_diets, carb_range, protein_range, fat_range):

    filtered_diets = []

    for diet in daily_diets:
        daily_total = diet.get('Daily Total', {})

        carbs = daily_total.get('Carbohydrate (g)', 0)
        protein = daily_total.get('Protein (g)', 0)
        fat = daily_total.get('Fat (g)', 0)
        calories = daily_total.get('Calories', 1)  # Avoid division by zero

        carb_ratio = (carbs * 4 / calories) * 100
        protein_ratio = (protein * 4 / calories) * 100
        fat_ratio = (fat * 9 / calories) * 100

        if (
            carb_range[0] <= carb_ratio <= carb_range[1]
            and protein_range[0] <= protein_ratio <= protein_range[1]
            and fat_range[0] <= fat_ratio <= fat_range[1]
        ):
            filtered_diets.append(diet)

    return filtered_diets

daily_diets = []
with open('/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/daily_diets_drop_duplicated_and_sampled_and_modified_nutri_facts.json', 'r') as f:
    for line in f:
        daily_diets.append(json.loads(line.strip()))

# +-15%
# carb_range = (35, 65)  # Carbohydrate ratio: 50%
# protein_range = (15, 45)  # Protein ratio: 30%
# fat_range = (5, 45)  # Fat ratio: 20%

carb_range = (40, 60)  # Carbohydrate ratio: 50%
protein_range = (20, 40)  # Protein ratio: 30%
fat_range = (10, 30)  # Fat ratio: 20%

filtered_diets = filter_diets_by_macros(daily_diets, carb_range, protein_range, fat_range)

with open('daily_diets_drop_duplicated_and_sampled_and_modified_nutri_facts_and_sampled.json', 'w') as f:
    json.dump(filtered_diets, f, indent=4)

print(f"Filtered {len(filtered_diets)} diets that meet the macronutrient ratio criteria.")


Filtered 685 diets that meet the macronutrient ratio criteria.


In [2]:
daily_diets = filtered_diets

In [3]:
len(daily_diets)

685

In [4]:
daily_diets[0]

{'Breakfast': {'Dish': 'Peach & Cream Smoothie',
  'Nutrition': {'Calories': '140',
   'Total Fat': {'Amount': '5g', 'Saturated Fat': '1g', 'Trans Fat': '0g'},
   'Cholesterol': '10mg',
   'Sodium': '95mg',
   'Total Carbohydrate': {'Amount': '19g',
    'Dietary Fiber': '4g',
    'Total Sugars': '8g',
    'Added Sugars': None},
   'Protein': '9g',
   'Potassium': '340mg'},
  'Ingredients': [{'label': 'Splenda®  Diabetes Care Vanilla Shake',
    'metric_measure': '1'},
   {'label': 'frozen peaches', 'metric_measure': '237 ml'},
   {'label': 'lemon(juice and zest)', 'metric_measure': '1/2 small'},
   {'label': 'pure vanilla extract', 'metric_measure': '5 g'},
   {'label': 'Splenda® Original Sweetener(optional)',
    'metric_measure': '2 packet'},
   {'label': 'sugar-free whipped cream', 'metric_measure': '30 ml'}]},
 'Lunch': {'Dish': 'Sugar Free Yogurt Parfait with Fresh Berries',
  'Nutrition': {'Calories': '180',
   'Total Fat': {'Amount': '1g', 'Saturated Fat': '0.3g', 'Trans Fat': '

In [5]:
# train/test split

import json
from sklearn.model_selection import train_test_split

# Split data into train and test (8:2 ratio)
daily_diets_train, daily_diets_test = train_test_split(daily_diets, test_size=0.2, random_state=42)

In [6]:
len(daily_diets_train)

548

In [7]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer

data = daily_diets_train

instruction_dataset = []

def custom_tokenizer(text):
    return text.split("\n")

def get_main_ingredients(ingredients_list):
    all_ingredients = [ingredient['label'] for meal in ingredients_list for ingredient in meal]
    all_ingredients_combined = "\n".join(all_ingredients)

    vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False)  
    tfidf_matrix = vectorizer.fit_transform([all_ingredients_combined])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.sum(axis=0).A1
    ingredient_scores = dict(zip(feature_names, scores))

    sorted_ingredients = sorted(ingredient_scores, key=ingredient_scores.get, reverse=True)
    return sorted_ingredients[:10]

for idx, day in enumerate(data):
    breakfast = day["Breakfast"]
    lunch = day["Lunch"]
    dinner = day["Dinner"]
    daily_nutrition = day["Daily Total"]

    ingredients_list = [
        breakfast["Ingredients"],
        lunch["Ingredients"],
        dinner["Ingredients"]
    ]

    main_ingredients = get_main_ingredients(ingredients_list)

    input_goals = (
        f"Ensure the daily carbohydrate intake does not exceed {daily_nutrition['Carbohydrate (g)']}g, "
        f"protein intake is at least {daily_nutrition['Protein (g)']}g, and fat intake does not exceed {daily_nutrition['Fat (g)']}g."
    )

    if idx % 10 == 0:
        input_goals = (
            f"Ensure the daily carbohydrate intake does not exceed {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein intake is at least {daily_nutrition['Protein (g)']}g, and fat intake does not exceed {daily_nutrition['Fat (g)']}g."
        )
        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. This meal plan is designed to maintain a balanced 50:30:20 macronutrient ratio throughout the day, supporting blood sugar stability and overall metabolic health. Carbohydrates are evenly distributed to prevent glucose spikes, while lean proteins and healthy fats enhance satiety. Processed sugars and refined carbs are minimized, with fiber-rich ingredients included for better digestion."
        })
    elif idx % 10 == 1:
        input_goals = (
            f"Ensure that the total carbohydrate intake does not go beyond {daily_nutrition['Carbohydrate (g)']}g, "
            f"maintain a minimum protein intake of {daily_nutrition['Protein (g)']}g, and restrict fat consumption to a maximum of {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output":
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan is structured to support stable blood sugar levels by distributing macronutrients in a 50:30:20 ratio. "
                    f"Carbohydrates are portioned throughout the day to avoid rapid glucose spikes, while lean proteins and healthy fats help sustain energy levels. "
                    f"Refined sugars and processed carbohydrates are minimized, and fiber-rich ingredients are prioritized for optimal digestion and metabolic health."
        })

    elif idx % 10 == 2:
        input_goals = (
            f"Maintain carbohydrate intake below {daily_nutrition['Carbohydrate (g)']}g, "
            f"consume at least {daily_nutrition['Protein (g)']}g of protein, and ensure that total fat intake does not surpass {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan follows a well-balanced macronutrient ratio of 50:30:20 to support sustained energy and blood sugar management. "
                    f"Carbohydrates are carefully distributed to prevent spikes, while lean protein and healthy fats contribute to overall satiety. "
                    f"Refined sugars and simple carbs are minimized, with an emphasis on fiber-rich foods for improved digestion and metabolic efficiency."
        })

    elif idx % 10 == 3:
        input_goals = (
            f"Keep carbohydrate intake under {daily_nutrition['Carbohydrate (g)']}g, "
            f"make sure to get no less than {daily_nutrition['Protein (g)']}g of protein, and limit fat intake to a maximum of {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",    
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan ensures a controlled macronutrient ratio of 50:30:20, helping to regulate blood sugar levels effectively. "
                    f"Carbohydrates are spaced out evenly throughout the day to prevent sudden fluctuations, while lean proteins and healthy fats "
                    f"aid in maintaining satiety and overall metabolic health. Processed sugars and refined carbohydrates are avoided, and fiber intake is optimized."
        })

    elif idx % 10 == 4:
        input_goals = (
            f"Ensure that daily carbohydrate intake remains below {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein consumption is at least {daily_nutrition['Protein (g)']}g, and fat intake stays within {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan follows a structured macronutrient ratio of 50:30:20 to aid in stabilizing blood sugar levels. "
                    f"Carbohydrates are evenly spaced throughout the meals to prevent sharp glucose spikes, while proteins and healthy fats "
                    f"provide sustained energy. Processed sugars and refined carbs are kept to a minimum, and fiber-rich foods support healthy digestion."
        })

    elif idx % 10 == 5:
        input_goals = (
            f"Restrict carbohydrate intake to no more than {daily_nutrition['Carbohydrate (g)']}g, "
            f"consume at least {daily_nutrition['Protein (g)']}g of protein, and limit fat consumption to {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"Following a 50:30:20 macronutrient balance, this meal plan is designed to maintain stable blood sugar levels. "
                    f"Carbohydrates are evenly distributed across meals to avoid glucose spikes, and proteins and healthy fats help prolong satiety. "
                    f"Refined sugars and processed carbohydrates are minimized, and fiber intake is emphasized for improved digestion and nutrient absorption."
        })

    elif idx % 10 == 6:
        input_goals = (
            f"Keep daily carbohydrate intake under {daily_nutrition['Carbohydrate (g)']}g, "
            f"ensure protein consumption meets or exceeds {daily_nutrition['Protein (g)']}g, and do not exceed {daily_nutrition['Fat (g)']}g of fat."
        )

        instruction_dataset.append({
            "instruction":  "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan follows a structured 50:30:20 macronutrient ratio to support stable glucose levels. "
                    f"Carbohydrates are portioned to prevent rapid blood sugar fluctuations, while lean protein and healthy fats "
                    f"ensure sustained energy and satiety. Processed sugars are minimized, and high-fiber foods are incorporated for better digestion."
        })

    elif idx % 10 == 7:
        input_goals = (
            f"Ensure that the total carbohydrate intake remains below {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein intake is at least {daily_nutrition['Protein (g)']}g, and fat intake does not exceed {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"Designed with a 50:30:20 macronutrient balance, this meal plan helps regulate blood sugar effectively. "
                    f"Carbohydrates are distributed strategically to avoid spikes, while proteins and healthy fats "
                    f"contribute to metabolic stability. Processed and refined carbohydrates are limited, and fiber-rich foods enhance digestion."
        })

    elif idx % 10 == 8:
        input_goals = (
            f"Limit carbohydrate intake to {daily_nutrition['Carbohydrate (g)']}g or less, "
            f"ensure protein intake is no lower than {daily_nutrition['Protein (g)']}g, and restrict fat consumption to {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"Following a structured 50:30:20 ratio, this meal plan optimizes blood sugar control by evenly spacing carbohydrate intake. "
                    f"Proteins and healthy fats support prolonged satiety and steady energy levels. Refined sugars and processed carbs are reduced, "
                    f"while fiber-rich foods play a key role in digestion and metabolic health."
        })
    elif idx % 10 == 9:
        input_goals = (
            f"Keep daily carbohydrate intake within {daily_nutrition['Carbohydrate (g)']}g, "
            f"make sure protein intake is at least {daily_nutrition['Protein (g)']}g, and "
            f"restrict total fat consumption to no more than {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. "
                    f"This meal plan follows a structured macronutrient balance of 50:30:20, ensuring an even distribution of carbohydrates "
                    f"to prevent blood sugar spikes. Lean proteins and healthy fats contribute to prolonged satiety and stable energy levels. "
                    f"Refined sugars and highly processed carbohydrates are minimized, while fiber-rich foods promote better digestion and metabolic efficiency."
        })


    selected_ingredients = random.sample(main_ingredients, min(1, len(main_ingredients)))

    for id, ingredient in enumerate(selected_ingredients):
        clean_ingredient = ingredient.replace("\\", "").replace("\"", "") 
        if id % 10 == 0: 
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "output": f"Breakfast: {breakfast['Dish'].replace("\\","").replace('\"', '')}. Lunch: {lunch['Dish'].replace("\\", "").replace('\"', '')}. Dinner: {dinner['Dish'].replace("\\", "").replace('\"', '')}. This plan is designed to maintain a 50:30:20 macronutrient ratio, balancing carbohydrates, proteins, and healthy fats to help regulate blood sugar levels. Carbohydrates are strategically portioned to prevent sudden glucose spikes, while fiber-rich foods improve digestion and promote steady energy levels throughout the day."
            })
        elif id % 10 == 1:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance and ensuring a steady distribution of nutrients throughout the day to prevent fluctuations in blood sugar levels.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan maintains a 50:30:20 macronutrient ratio, ensuring that carbohydrates are evenly spread throughout the day to prevent sharp glucose fluctuations. It incorporates lean proteins and healthy fats to sustain energy levels while emphasizing fiber-rich foods for improved digestion and metabolic health."
            })
        elif id % 10 == 2:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while incorporating a diverse range of nutrient-dense foods to optimize blood sugar control.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. Following a 50:30:20 macronutrient balance, this meal plan is designed to distribute carbohydrates steadily throughout the day, reducing the risk of blood sugar spikes. It includes nutrient-dense proteins and healthy fats to promote satiety, while fiber-rich ingredients support digestive health and overall metabolism."
            })
        elif id % 10 == 3:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while focusing on foods that provide long-lasting energy and minimize post-meal glucose spikes.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. With a structured 50:30:20 macronutrient approach, this meal plan ensures carbohydrates are strategically portioned to maintain stable blood sugar levels. High-quality proteins and healthy fats are incorporated to prolong satiety, while whole foods rich in fiber contribute to optimal digestion and metabolic function."

            })
        elif id % 10 == 4:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while ensuring a gradual release of glucose for sustained metabolic health.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan adheres to a 50:30:20 macronutrient framework, emphasizing balanced carbohydrate intake to prevent glucose surges. Lean protein sources and unsaturated fats enhance satiety and energy stability, while fiber-dense foods aid in digestion and metabolic efficiency."

            })

        elif id % 10 == 5:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, Following a 50:30:20 macronutrient balance, this plan ensures an even distribution of nutrients throughout the day, helping to regulate blood sugar levels and maintain metabolic stability.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. Designed to align with a 50:30:20 macronutrient ratio, this meal plan distributes carbohydrates in a controlled manner to support blood sugar regulation. It includes lean proteins and beneficial fats for sustained energy, while fiber-rich meals promote gut health and steady digestion."

            })
        elif id % 10 == 6:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, By adhering to a 50:30:20 macronutrient balance, this meal plan promotes a steady release of nutrients, preventing blood sugar fluctuations and supporting overall metabolic health.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. By maintaining a 50:30:20 macronutrient split, this meal plan ensures an even distribution of carbohydrates to prevent rapid spikes in blood sugar. It integrates quality proteins and essential fats to support long-lasting fullness, while whole-food sources of fiber enhance digestion and metabolic balance."

            })
        elif id % 10 == 7:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while selecting ingredients that optimize insulin sensitivity.", 
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan follows a well-balanced 50:30:20 macronutrient composition, ensuring carbohydrates are spread throughout the day to regulate blood glucose levels. Lean proteins and plant-based fats provide steady energy, while high-fiber meals support digestion and metabolic health."

            })
        elif id % 10 == 8:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance and reducing reliance on high-glycemic foods to prevent rapid sugar fluctuations.", 
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. With a carefully structured 50:30:20 macronutrient balance, this meal plan focuses on stable carbohydrate distribution to manage blood sugar levels effectively. It includes sufficient protein and healthy fats for prolonged satiety, while fiber-rich ingredients contribute to optimal digestion and metabolism."

            })
        elif id % 10 == 9:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, mMaintaining a 50:30:20 macronutrient distribution, this plan ensures a well-regulated intake of nutrients throughout the day to stabilize glucose levels and enhance energy consistency.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan adheres to a structured 50:30:20 macronutrient ratio, balancing carbohydrates to prevent glucose fluctuations. The inclusion of lean proteins and beneficial fats helps sustain energy levels, while an emphasis on fiber-rich foods supports digestive health and metabolic efficiency."

            })

    # instruction_dataset.append({
    #     "instruction": "Analyze and summarize the nutritional content of a daily diet.",
    #     "input": {
    #         "Breakfast": breakfast["Dish"],
    #         "Lunch": lunch["Dish"],
    #         "Dinner": dinner["Dish"]
    #     },
    #     "output": {
    #         "Daily Total": daily_nutrition
    #     }
    # })

with open("daily_diets_drop_duplicated_and_sampled_train_instruction_dataset.json", "w", encoding="utf-8") as f:
    json.dump(instruction_dataset, f, ensure_ascii=False, indent=4)

print(f"Generated {len(instruction_dataset)} instruction items.")


/data/jaesung/anaconda3/envs/meal/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Generated 1096 instruction items.


In [8]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer

data = daily_diets_test

instruction_dataset = []

def custom_tokenizer(text):
    return text.split("\n")

def get_main_ingredients(ingredients_list):
    all_ingredients = [ingredient['label'] for meal in ingredients_list for ingredient in meal]
    all_ingredients_combined = "\n".join(all_ingredients)

    vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False)  
    tfidf_matrix = vectorizer.fit_transform([all_ingredients_combined])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.sum(axis=0).A1
    ingredient_scores = dict(zip(feature_names, scores))

    sorted_ingredients = sorted(ingredient_scores, key=ingredient_scores.get, reverse=True)
    return sorted_ingredients[:10]

for idx, day in enumerate(data):
    breakfast = day["Breakfast"]
    lunch = day["Lunch"]
    dinner = day["Dinner"]
    daily_nutrition = day["Daily Total"]

    ingredients_list = [
        breakfast["Ingredients"],
        lunch["Ingredients"],
        dinner["Ingredients"]
    ]

    main_ingredients = get_main_ingredients(ingredients_list)

    if idx % 10 == 0:
        input_goals = (
            f"Ensure the daily carbohydrate intake does not exceed {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein intake is at least {daily_nutrition['Protein (g)']}g, and fat intake does not exceed {daily_nutrition['Fat (g)']}g."
        )
        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}. This meal plan is designed to maintain a balanced 50:30:20 macronutrient ratio throughout the day, supporting blood sugar stability and overall metabolic health. Carbohydrates are evenly distributed to prevent glucose spikes, while lean proteins and healthy fats enhance satiety. Processed sugars and refined carbs are minimized, with fiber-rich ingredients included for better digestion."
        })
    elif idx % 10 == 1:
        input_goals = (
            f"Ensure that the total carbohydrate intake does not go beyond {daily_nutrition['Carbohydrate (g)']}g, "
            f"maintain a minimum protein intake of {daily_nutrition['Protein (g)']}g, and restrict fat consumption to a maximum of {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output":
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan is structured to support stable blood sugar levels by distributing macronutrients in a 50:30:20 ratio. "
                    f"Carbohydrates are portioned throughout the day to avoid rapid glucose spikes, while lean proteins and healthy fats help sustain energy levels. "
                    f"Refined sugars and processed carbohydrates are minimized, and fiber-rich ingredients are prioritized for optimal digestion and metabolic health."
        })

    elif idx % 10 == 2:
        input_goals = (
            f"Maintain carbohydrate intake below {daily_nutrition['Carbohydrate (g)']}g, "
            f"consume at least {daily_nutrition['Protein (g)']}g of protein, and ensure that total fat intake does not surpass {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan follows a well-balanced macronutrient ratio of 50:30:20 to support sustained energy and blood sugar management. "
                    f"Carbohydrates are carefully distributed to prevent spikes, while lean protein and healthy fats contribute to overall satiety. "
                    f"Refined sugars and simple carbs are minimized, with an emphasis on fiber-rich foods for improved digestion and metabolic efficiency."
        })

    elif idx % 10 == 3:
        input_goals = (
            f"Keep carbohydrate intake under {daily_nutrition['Carbohydrate (g)']}g, "
            f"make sure to get no less than {daily_nutrition['Protein (g)']}g of protein, and limit fat intake to a maximum of {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",        
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan ensures a controlled macronutrient ratio of 50:30:20, helping to regulate blood sugar levels effectively. "
                    f"Carbohydrates are spaced out evenly throughout the day to prevent sudden fluctuations, while lean proteins and healthy fats "
                    f"aid in maintaining satiety and overall metabolic health. Processed sugars and refined carbohydrates are avoided, and fiber intake is optimized."
        })

    elif idx % 10 == 4:
        input_goals = (
            f"Ensure that daily carbohydrate intake remains below {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein consumption is at least {daily_nutrition['Protein (g)']}g, and fat intake stays within {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan follows a structured macronutrient ratio of 50:30:20 to aid in stabilizing blood sugar levels. "
                    f"Carbohydrates are evenly spaced throughout the meals to prevent sharp glucose spikes, while proteins and healthy fats "
                    f"provide sustained energy. Processed sugars and refined carbs are kept to a minimum, and fiber-rich foods support healthy digestion."
        })

    elif idx % 10 == 5:
        input_goals = (
            f"Restrict carbohydrate intake to no more than {daily_nutrition['Carbohydrate (g)']}g, "
            f"consume at least {daily_nutrition['Protein (g)']}g of protein, and limit fat consumption to {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"Following a 50:30:20 macronutrient balance, this meal plan is designed to maintain stable blood sugar levels. "
                    f"Carbohydrates are evenly distributed across meals to avoid glucose spikes, and proteins and healthy fats help prolong satiety. "
                    f"Refined sugars and processed carbohydrates are minimized, and fiber intake is emphasized for improved digestion and nutrient absorption."
        })

    elif idx % 10 == 6:
        input_goals = (
            f"Keep daily carbohydrate intake under {daily_nutrition['Carbohydrate (g)']}g, "
            f"ensure protein consumption meets or exceeds {daily_nutrition['Protein (g)']}g, and do not exceed {daily_nutrition['Fat (g)']}g of fat."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan follows a structured 50:30:20 macronutrient ratio to support stable glucose levels. "
                    f"Carbohydrates are portioned to prevent rapid blood sugar fluctuations, while lean protein and healthy fats "
                    f"ensure sustained energy and satiety. Processed sugars are minimized, and high-fiber foods are incorporated for better digestion."
        })

    elif idx % 10 == 7:
        input_goals = (
            f"Ensure that the total carbohydrate intake remains below {daily_nutrition['Carbohydrate (g)']}g, "
            f"protein intake is at least {daily_nutrition['Protein (g)']}g, and fat intake does not exceed {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"Designed with a 50:30:20 macronutrient balance, this meal plan helps regulate blood sugar effectively. "
                    f"Carbohydrates are distributed strategically to avoid spikes, while proteins and healthy fats "
                    f"contribute to metabolic stability. Processed and refined carbohydrates are limited, and fiber-rich foods enhance digestion."
        })

    elif idx % 10 == 8:
        input_goals = (
            f"Limit carbohydrate intake to {daily_nutrition['Carbohydrate (g)']}g or less, "
            f"ensure protein intake is no lower than {daily_nutrition['Protein (g)']}g, and restrict fat consumption to {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"Following a structured 50:30:20 ratio, this meal plan optimizes blood sugar control by evenly spacing carbohydrate intake. "
                    f"Proteins and healthy fats support prolonged satiety and steady energy levels. Refined sugars and processed carbs are reduced, "
                    f"while fiber-rich foods play a key role in digestion and metabolic health."
        })
    elif idx % 10 == 9:
        input_goals = (
            f"Keep daily carbohydrate intake within {daily_nutrition['Carbohydrate (g)']}g, "
            f"make sure protein intake is at least {daily_nutrition['Protein (g)']}g, and "
            f"restrict total fat consumption to no more than {daily_nutrition['Fat (g)']}g."
        )

        instruction_dataset.append({
            "instruction": "Generate a structured daily meal plan for diabetes management. The output must strictly follow this format:\n\n"
                   "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
                   "The recommended meals must be selected only from the provided dataset and should not include any additional information. "
                   "Then, provide a brief explanation of how the selected meals contribute to diabetes management. "
                   "The explanation must come after the meals and should be no longer than three sentences.",
            "input": input_goals,
            "output": 
                    f"Breakfast: {breakfast['Dish'].replace("\\","").replace("\"", "")}. Lunch: {lunch['Dish'].replace("\\","").replace("\"", "")}. Dinner: {dinner['Dish'].replace("\\","").replace("\"", "")}."
                    f"This meal plan follows a structured macronutrient balance of 50:30:20, ensuring an even distribution of carbohydrates "
                    f"to prevent blood sugar spikes. Lean proteins and healthy fats contribute to prolonged satiety and stable energy levels. "
                    f"Refined sugars and highly processed carbohydrates are minimized, while fiber-rich foods promote better digestion and metabolic efficiency."
        })

    selected_ingredients = random.sample(main_ingredients, min(1, len(main_ingredients)))

    for id, ingredient in enumerate(selected_ingredients):
        clean_ingredient = ingredient.replace("\\", "").replace("\"", "")
        if id % 10 == 0: 
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance and supporting blood sugar stability for individuals with diabetes.",
                "output": f"Breakfast: {breakfast['Dish'].replace("\\","").replace('\"', '')}. Lunch: {lunch['Dish'].replace("\\", "").replace('\"', '')}. Dinner: {dinner['Dish'].replace("\\", "").replace('\"', '')}. This plan is designed to maintain a 50:30:20 macronutrient ratio, balancing carbohydrates, proteins, and healthy fats to help regulate blood sugar levels. Carbohydrates are strategically portioned to prevent sudden glucose spikes, while fiber-rich foods improve digestion and promote steady energy levels throughout the day."
            })
        elif id % 10 == 1:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance and ensuring a steady distribution of nutrients throughout the day to prevent fluctuations in blood sugar levels.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan maintains a 50:30:20 macronutrient ratio, ensuring that carbohydrates are evenly spread throughout the day to prevent sharp glucose fluctuations. It incorporates lean proteins and healthy fats to sustain energy levels while emphasizing fiber-rich foods for improved digestion and metabolic health."
            })
        elif id % 10 == 2:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while incorporating a diverse range of nutrient-dense foods to optimize blood sugar control.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. Following a 50:30:20 macronutrient balance, this meal plan is designed to distribute carbohydrates steadily throughout the day, reducing the risk of blood sugar spikes. It includes nutrient-dense proteins and healthy fats to promote satiety, while fiber-rich ingredients support digestive health and overall metabolism."
            })
        elif id % 10 == 3:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while focusing on foods that provide long-lasting energy and minimize post-meal glucose spikes.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. With a structured 50:30:20 macronutrient approach, this meal plan ensures carbohydrates are strategically portioned to maintain stable blood sugar levels. High-quality proteins and healthy fats are incorporated to prolong satiety, while whole foods rich in fiber contribute to optimal digestion and metabolic function."

            })
        elif id % 10 == 4:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while ensuring a gradual release of glucose for sustained metabolic health.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan adheres to a 50:30:20 macronutrient framework, emphasizing balanced carbohydrate intake to prevent glucose surges. Lean protein sources and unsaturated fats enhance satiety and energy stability, while fiber-dense foods aid in digestion and metabolic efficiency."

            })

        elif id % 10 == 5:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, Following a 50:30:20 macronutrient balance, this plan ensures an even distribution of nutrients throughout the day, helping to regulate blood sugar levels and maintain metabolic stability.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. Designed to align with a 50:30:20 macronutrient ratio, this meal plan distributes carbohydrates in a controlled manner to support blood sugar regulation. It includes lean proteins and beneficial fats for sustained energy, while fiber-rich meals promote gut health and steady digestion."

            })
        elif id % 10 == 6:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, By adhering to a 50:30:20 macronutrient balance, this meal plan promotes a steady release of nutrients, preventing blood sugar fluctuations and supporting overall metabolic health.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. By maintaining a 50:30:20 macronutrient split, this meal plan ensures an even distribution of carbohydrates to prevent rapid spikes in blood sugar. It integrates quality proteins and essential fats to support long-lasting fullness, while whole-food sources of fiber enhance digestion and metabolic balance."

            })
        elif id % 10 == 7:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance while selecting ingredients that optimize insulin sensitivity.", 
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan follows a well-balanced 50:30:20 macronutrient composition, ensuring carbohydrates are spread throughout the day to regulate blood glucose levels. Lean proteins and plant-based fats provide steady energy, while high-fiber meals support digestion and metabolic health."

            })
        elif id % 10 == 8:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, maintaining a 50:30:20 macronutrient balance and reducing reliance on high-glycemic foods to prevent rapid sugar fluctuations.", 
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. With a carefully structured 50:30:20 macronutrient balance, this meal plan focuses on stable carbohydrate distribution to manage blood sugar levels effectively. It includes sufficient protein and healthy fats for prolonged satiety, while fiber-rich ingredients contribute to optimal digestion and metabolism."

            })
        elif id % 10 == 9:
            instruction_dataset.append({
                "instruction": "Generate a structured daily meal plan for diabetes management. The recommended meals must be selected only from the provided dataset and should not include any additional information.\n\n"
               "The output must strictly follow this format:\n\n"
               "Breakfast: [meal]\nLunch: [meal]\nDinner: [meal]\n\n"
               "Then, provide a brief explanation of how the selected meals support blood sugar control. "
               "The explanation should be concise and relevant to diabetes management, focusing on how the selected meals balance macronutrients.",
                "input_text": f"Create a meal plan that includes {clean_ingredient}, mMaintaining a 50:30:20 macronutrient distribution, this plan ensures a well-regulated intake of nutrients throughout the day to stabilize glucose levels and enhance energy consistency.",
                "output_text": f"Breakfast: {breakfast['Dish'].replace('\\', '').replace('\"', '')}. Lunch: {lunch['Dish'].replace('\\', '').replace('\"', '')}. Dinner: {dinner['Dish'].replace('\\', '').replace('\"', '')}. This meal plan adheres to a structured 50:30:20 macronutrient ratio, balancing carbohydrates to prevent glucose fluctuations. The inclusion of lean proteins and beneficial fats helps sustain energy levels, while an emphasis on fiber-rich foods supports digestive health and metabolic efficiency."

            })
    # instruction_dataset.append({
    #     "instruction": "Analyze and summarize the nutritional content of a daily diet.",
    #     "input": {
    #         "Breakfast": breakfast["Dish"],
    #         "Lunch": lunch["Dish"],
    #         "Dinner": dinner["Dish"]
    #     },
    #     "output": {
    #         "Daily Total": daily_nutrition
    #     }
    # })

with open("daily_diets_drop_duplicated_and_sampled_test_instruction_dataset.json", "w", encoding="utf-8") as f:
    json.dump(instruction_dataset, f, ensure_ascii=False, indent=4)

print(f"Generated {len(instruction_dataset)} instruction items.")


Generated 274 instruction items.
